In [2]:
import ee
import hvplot.xarray
import xarray

import rasterio as rio
import rioxarray

import xarray
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path
from tqdm.notebook import tqdm
import math
import json
from datetime import datetime

In [3]:
dfs = []

for shp in Path('data/ML_training_labels/retrogressive_thaw_slumps/').glob('*/*.shp'):
  dfs.append(gpd.read_file(shp))
  
gdf = pd.concat(dfs).reset_index(drop=True)

# The `geedim` way

Best approach so far – TODO: Parallelization

In [33]:
import geedim as gd
gd.Initialize()

BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']

lookback  = pd.Timedelta(days=5)
lookahead = pd.Timedelta(days=5)

s2 = gd.MaskedCollection.from_name("COPERNICUS/S2_HARMONIZED")

In [34]:
for (image_id, image_date, region), shapes in tqdm(gdf.groupby(['image_id', 'image_date', 'region'])):
  date = pd.to_datetime(image_date) 

  # Calculate UTM Zone from the region centroid
  centroid_x = 0.5 * (shapes.bounds.minx.min() + shapes.bounds.maxx.max())
  UTMZone = int(math.ceil((180 + centroid_x) / 6))
  CRS = f'EPSG:326{UTMZone:02d}'
  
  # Define ROI as a ee Geometry
  shapes = shapes.to_crs(CRS)
  bounds = shapes.bounds
  bounds = ee.Geometry.Rectangle(
    # xMin, yMin, xMax, yMax
    [bounds.minx.min(), bounds.miny.min(),
     bounds.maxx.max(), bounds.maxy.max()],
    proj=CRS, geodesic=True, evenOdd=False)

  
  imgs = s2.search(
    start_date=date - lookback,  # Select starting `lookback` days before PSOrthoTile
    end_date=date + lookahead,   # Select until `lookahead` days after PSOrthoTile
    region=bounds,
    fill_portion=90,             # Ensure at least 90% of pixels are valid
    custom_filter='CLOUDY_PIXEL_PERCENTAGE < 20'  # Filter Scenes with too many clouds
  )
  
  print(f'Found {len(imgs.properties)} scenes for {image_id}')
  for img in imgs.properties:
    s2_id = img.split('/')[-1]
    outpath = Path(f'data/Sentinel2/{image_id}/{s2_id}.tif')
    outpath.parent.mkdir(parents=True, exist_ok=True)
    
    # By default, geedim downloads scenes at native resolution and CRS
    gd.MaskedImage.from_id(img).download(outpath, region=bounds)
    
    # Save S2 image metadata, we might need that later
    with outpath.with_suffix('.json').open('w') as f:
      json.dump(imgs.properties[img], f)

  0%|          | 0/154 [00:00<?, ?it/s]

Found 3 scenes for 20180702_025400_0f31


20180627T035541_20180627T035537_T51WWS.tif: |           | 0.00/40.2M (raw) [  0.0%] in 00:00 (eta:     ?)

20180702T035539_20180702T035535_T51WWS.tif: |           | 0.00/40.2M (raw) [  0.0%] in 00:00 (eta:     ?)

20180704T034541_20180704T034535_T51WWS.tif: |           | 0.00/40.2M (raw) [  0.0%] in 00:00 (eta:     ?)

...

Found 1 scenes for 4751812_3973218_2021-08-01_1026


20210801T091559_20210801T091554_T39XWB.tif: |            | 0.00/155M (raw) [  0.0%] in 00:00 (eta:     ?)

Found 2 scenes for 4770635_4572716_2021-08-06_2416


20210805T062631_20210805T062631_T45XWA.tif: |            | 0.00/238M (raw) [  0.0%] in 00:00 (eta:     ?)

20210806T064629_20210806T064625_T45XWA.tif: |            | 0.00/238M (raw) [  0.0%] in 00:00 (eta:     ?)

# The vanilla `ee` way

Works, but is kind of clumsy with regards to getting the data in and out of GDrive.

In [4]:
ee.Initialize()

DRIVE_FOLDER = 'RTS_S2'
BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']

s2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")

lookback  = pd.Timedelta(days=14)
lookahead = pd.Timedelta(days=7)

tasklist = []

for (image_id, image_date, region), shapes in tqdm(gdf.groupby(['image_id', 'image_date', 'region'])): 
  centroid_x = 0.5 * (shapes.bounds.minx.min() + shapes.bounds.maxx.max())
  UTMZone = int(math.ceil((180 + centroid_x) / 6))
  CRS = f'EPSG:326{UTMZone:02d}'
  
  shapes = shapes.to_crs(CRS)
  bounds = shapes.bounds
  bounds = ee.Geometry.Rectangle(
    # xMin, yMin, xMax, yMax
    [bounds.minx.min(), bounds.miny.min(),
     bounds.maxx.max(), bounds.maxy.max()],
    proj=CRS, geodesic=True, evenOdd=False)
  
  date = pd.to_datetime(image_date)
  
  imgs = s2
  imgs = imgs.filterDate(date-lookback, date+lookahead)
  imgs = imgs.filterBounds(bounds)
  # imgs = imgs.filter(ee.Filter.intersects(leftField='.all', rightValue=ee.Feature(bounds)))
  imgs = imgs.filter(ee.Filter.contains(leftField='.all', rightValue=ee.Feature(bounds)))
  imgs = imgs.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
  imgs = imgs.select(BANDS)
  imgs = imgs.map(lambda x: x.clip(bounds))
  
  total = imgs.size().getInfo()
  print(f'{image_id}: {total}')
  imgs = imgs.toList(total)

  for i in range(total):
    img = ee.Image(imgs.get(i))
    metadata = img.toDictionary().getInfo()

    timestamp = datetime.fromtimestamp(img.date().getInfo()['value'] / 1000)
    day = timestamp.isoformat().split('T')[0]

    filename = f'{image_id}_{day}'

    task = ee.batch.Export.image.toDrive(
        image=img,
        description=filename,
        scale=10,
        region=bounds,
        fileNamePrefix=filename,
        folder=DRIVE_FOLDER,
        skipEmptyTiles=True,
        crs=CRS,
    )
    task.start()
    tasklist.append(task)
    
    with open(f'data/Sentinel2/{filename}.json', 'w') as f:
      json.dump(metadata, f)
      
  print(f"Done with {image_id}")
  break

  0%|          | 0/154 [00:00<?, ?it/s]

20180702_025400_0f31: 6
Done with 20180702_025400_0f31


# The `wxee` way

We seem to get lots of `nan` bands this way... Not sure why. Let's not use wxee.

In [ ]:
import wxee
wxee.Initialize()

BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']

s2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")

lookback  = pd.Timedelta(days=14)
lookahead = pd.Timedelta(days=7)

id2paths = {}

for (image_id, image_date, region), shapes in tqdm(gdf.groupby(['image_id', 'image_date', 'region'])):
  if Path(f'data/Sentinel2/{image_id}/').exists():
    # Skip already downloaded tiles
    continue
  
  centroid_x = 0.5 * (shapes.bounds.minx.min() + shapes.bounds.maxx.max())
  UTMZone = int(math.ceil((180 + centroid_x) / 6))
  CRS = f'EPSG:326{UTMZone:02d}'
  
  shapes = shapes.to_crs(CRS)
  bounds = shapes.bounds  
  bounds = ee.Geometry.Rectangle(
    # xMin, yMin, xMax, yMax
    [bounds.minx.min(), bounds.miny.min(),
     bounds.maxx.max(), bounds.maxy.max()],
    proj=CRS, geodesic=True, evenOdd=False)
  
  date = pd.to_datetime(image_date)
  
  imgs = s2
  imgs = imgs.filterDate(date-lookback, date+lookahead)
  imgs = imgs.filterBounds(bounds)
  # imgs = imgs.filter(ee.Filter.intersects(leftField='.all', rightValue=ee.Feature(bounds)))
  imgs = imgs.filter(ee.Filter.contains(leftField='.all', rightValue=ee.Feature(bounds)))
  imgs = imgs.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
  imgs = imgs.map(lambda x: x.clip(bounds))
   
  # acquisition_times = imgs.aggregate_array("system:time_start").getInfo()
  # closest = np.argmin(np.abs(np.asarray(acquisition_times) - date.value / 1e6))
  # img = imgs.toList(imgs, len(acquisition_times)).get(closest)
  
  try:
    data = imgs.wx.to_xarray(scale=10, crs=CRS)
  except ee.EEException:
    print(f'Got EEException for {image_id}, retrying bandwise...')
    data = []
    for band in BANDS:
      data.append(imgs.select([band]).wx.to_xarray(scale=10, crs=CRS))
    data = xarray.merge(data)
    
  if not data.dims:
    # Empty data...
    continue
  
  for time, image in data.groupby('time'):
    day = str(time).split('T')[0]
    
    isnan = np.isnan(image).all()
    if all(isnan[b] for b in isnan):
      continue
    
    outpath = Path(f'data/Sentinel2/{image_id}/{day}.tif')
    outpath.parent.mkdir(parents=True, exist_ok=True)
    image.rio.to_raster(outpath)
  # break
    
  # id2paths[image_id] = paths